In [84]:
# Importing necessary libraries and importing CSV files into respective dataframes

import pandas as pd

df_CPI = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/CPI-inflation-by-region-1914-202.csv")
df_Housing = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/housing-supply-price-rental.csv")
df_HPIRegions = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/HPI 1981-2022 by regions.csv")
df_HPIFederalandProvincial = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/HPI 1981-2022 federal and provincial.csv")
df_Income = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/income-distribution-2012-2020.csv")
df_Interest = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/Interest and mortgage rates 1951-2022.csv")
df_PopulationDwellings = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/population_dwellings_count.csv")
df_PopulationRegion = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/population-by-region-1946-2022.csv")
df_Structural = pd.read_csv("/workspaces/MSCI466_Project/data/housing-affordability-in-canada/Structural-dwellings-household-size.csv")

In [42]:


df_Interest.head(10)

,Date,Mortgage Rate,Interest Rate
0,1951-01,5.00,0.0
1,1951-02,5.00,0.0
2,1951-03,5.00,0.0
3,1951-04,5.25,0.0
4,1951-05,5.50,0.0
5,1951-06,5.50,0.0
6,1951-07,5.62,0.0
7,1951-08,5.62,0.0
8,1951-09,5.75,0.0
9,1951-10,5.75,0.0


In [85]:
# Deletes regions other than Canada
df_PopulationRegion = df_PopulationRegion[df_PopulationRegion['COORDINATE'] == 1]
df_PopulationRegion.drop(columns=['COORDINATE'], inplace=True)

# Converts REF_DATE to standard format
df_PopulationRegion['REF_DATE'] = pd.to_datetime(df_PopulationRegion['REF_DATE'], format='%b-%y')
def adjust_year(date):
    if date.year < 25:
        return date.replace(year=date.year)
    elif date.year >= 25 and date.year <= 99:
        return date.replace(year=date.year - 1000)
    else:
        return date
df_PopulationRegion['REF_DATE'] = df_PopulationRegion['REF_DATE'].apply(adjust_year)
df_PopulationRegion['REF_DATE'] = df_PopulationRegion['REF_DATE'].dt.strftime('%Y-%m')

df_PopulationRegion.head(306)

,REF_DATE,GEO,DGUID,VECTOR,Population estimate
0,2046-01,Canada,2016A000011124,v1,12188000
1,2046-04,Canada,2016A000011124,v1,12241000
2,2046-07,Canada,2016A000011124,v1,12316000
3,2046-10,Canada,2016A000011124,v1,12393000
4,2047-01,Canada,2016A000011124,v1,12450000
...,...,...,...,...,...
3769,2021-04,Canada,2016A000011124,v1,38153211
3783,2021-07,Canada,2016A000011124,v1,38246108
3797,2021-10,Canada,2016A000011124,v1,38436447
3811,2022-01,Canada,2016A000011124,v1,38526760
